# Synthesize Audio

## Environment

The sample is using key authentication and is loading the necessary information (api key, endpoint etc.) from a centralized configuration file.
The configuration file is created using [setup.azcli](../setup/setup.azcli).


In [10]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.43.0"
#r "nuget: DotNetEnv, 3.1.1"

using DotNetEnv;
using System.IO;
using Microsoft.CognitiveServices.Speech;
using Microsoft.CognitiveServices.Speech.Audio;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("CNV_SPEECH_APIKEY");
string region = Env.GetString("CNV_SPEECH_REGION");
string customModelEndpointId = Env.GetString("CNV_CUSTOMMODEL_ID");
string customSpeechSynthesisVoiceName = Env.GetString("CNV_CUSTOMMODEL_NAME");

Console.WriteLine($"Configuration loaded...");


Installed Packages DotNetEnv, 3.1.1 Microsoft.CognitiveServices.Speech, 1.43.0

Configuration loaded...


## Input / Output

Text which should be synthesized is loaded from the [../assets/input](../assets/input/) folder.
Output files are defined which are stored in [../assets/output](../assets/output/) folder.

In [ ]:
string inputTextFile = "../assets/input/input_robert.txt";
string textContent = File.ReadAllText(inputTextFile);

string outputAudioFile = "../assets/output/output_robert.wav";
string outputAudioFileStream = "../assets/output/output_robert_stream.wav";

Console.WriteLine($"Text content loaded...");
Console.WriteLine($"Output audio files defied...");

Text content loaded...
Output audio files defied...


## Synthesize text

A custom neural voice model is used to synthesize the provided text.


In [15]:
SpeechConfig speechConfig = SpeechConfig.FromSubscription(apiKey, region);

// custom voice model
speechConfig.EndpointId = customModelEndpointId;
speechConfig.SpeechSynthesisVoiceName = customSpeechSynthesisVoiceName;

Console.WriteLine($"Custom voice model: {speechConfig.SpeechSynthesisVoiceName}...");

Custom voice model: RobEichCNVLite-English...


### Batch

File content is synthesized in one batch.

In [16]:
using (AudioConfig audioConfig = AudioConfig.FromWavFileOutput(outputAudioFile)) {
    using (SpeechSynthesizer speechSynthesizer = new SpeechSynthesizer(speechConfig, audioConfig)) {
        using (SpeechSynthesisResult speechSynthesisResult = await speechSynthesizer.SpeakTextAsync(textContent)) {
            if (speechSynthesisResult.Reason == ResultReason.SynthesizingAudioCompleted)
            {
                Console.WriteLine($"Speech synthesizing: {speechSynthesisResult.Reason}");
            }
        }    
    };
};

Console.WriteLine($"Audio file created: {outputAudioFile}");

Speech synthesizing: SynthesizingAudioCompleted
Audio file created: ../assets/output/output_robert.wav


### Stream

Single lines (sentences) are synthesized.

In [18]:
using (AudioConfig audioConfig = AudioConfig.FromWavFileOutput(outputAudioFileStream))
using (SpeechSynthesizer speechSynthesizer = new SpeechSynthesizer(speechConfig, audioConfig)) {

    TaskCompletionSource<bool> synthesisCompletionSource = new TaskCompletionSource<bool>();
    speechSynthesizer.SynthesisCompleted += (s, e) =>
    {
        synthesisCompletionSource.TrySetResult(true);
    };
    speechSynthesizer.SynthesisCanceled += (s, e) =>
    {
        synthesisCompletionSource.TrySetResult(true);
    };

    using (MemoryStream memoryStream = new MemoryStream(Encoding.UTF8.GetBytes(textContent)))
    using (StreamReader reader = new StreamReader(memoryStream))
    {
        string? textFromStream = "---START---";
        while (!String.IsNullOrEmpty(textFromStream))
        {
            textFromStream = await reader.ReadLineAsync();
            if (String.IsNullOrEmpty(textFromStream)) break;
            
            Console.WriteLine($"Text from stream: {textFromStream}");
            await speechSynthesizer.StartSpeakingTextAsync(textFromStream);
            
            // Wait for the synthesis process to complete
            bool synthesisSucceeded = await synthesisCompletionSource.Task;
            if (!synthesisSucceeded)
            {
                Console.WriteLine("Synthesis failed or was canceled.");
                break;
            }

            // Reset the TaskCompletionSource for the next synthesis
            synthesisCompletionSource = new TaskCompletionSource<bool>();

        }
    }
}

Console.WriteLine($"Audio file created: {outputAudioFileStream}");


Text from stream: Hello everyone,
Text from stream: My name is Robert, 
Text from stream: and I am excited to welcome you to the key note of this exciting event!
Audio file created: ../assets/output/output_robert_stream.wav
